In [2]:
import os
os.chdir("../")
%pwd

'd:\\Chicken-Disease-Classification'

In [58]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [59]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import *
import tensorflow as tf

class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self):

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config





In [60]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [61]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,  # Include file extension here
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [62]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def compile_model(self):
        # Compile the model with the same optimizer and loss function
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def reset_optimizer_state(self):
        # Reset the optimizer state
        for layer in self.model.layers:
            if hasattr(layer, 'optimizer'):
                for optimizer in layer.optimizer:
                    optimizer.reset_states()



    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Recompile the model before training
        self.compile_model()

        # Reset optimizer state
        self.reset_optimizer_state()

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [63]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

[2024-03-25 22:03:39,755: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-25 22:03:39,770: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-25 22:03:39,771: INFO: common: created directory at: artifacts]
[2024-03-25 22:03:39,772: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-03-25 22:03:39,773: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-03-25 22:03:39,774: INFO: common: created directory at: artifacts\training]


[2024-03-25 22:03:41,187: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 147s 8s/step - accuracy: 0.5866 - loss: 3.3231 - val_accuracy: 0.6250 - val_loss: 0.6547
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 1:58 7s/step - accuracy: 0.6875 - loss: 0.6466

c:\Users\nikhi\Anaconda3\envs\chickendiseaseproject\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 201ms/step - accuracy: 0.6875 - loss: 0.3403 - val_accuracy: 0.1429 - val_loss: 0.3587
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 133s 7s/step - accuracy: 0.5854 - loss: 0.6429 - val_accuracy: 0.7344 - val_loss: 0.6254
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5000 - loss: 0.4471 - val_accuracy: 0.8571 - val_loss: 0.2519
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 153s 8s/step - accuracy: 0.8788 - loss: 0.2820 - val_accuracy: 0.9375 - val_loss: 0.1597
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 18s 837ms/step - accuracy: 0.8750 - loss: 0.1065 - val_accuracy: 1.0000 - val_loss: 0.0090
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 131s 7s/step - accuracy: 0.9426 - loss: 0.1324 - val_accuracy: 0.9688 - val_loss: 0.0811
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 81ms/step - accuracy: 0.9375 - loss: 0.0462 - val_accuracy: 0.9286 - val_loss: 0.0511
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 138s 7s/step - accuracy: 0.9235 - loss: 0.2147 - val_accuracy: 0.9375 - val_loss: